# Progetto Marena — Gruppo 7
## Problema 2 + Modello CGMYB: Metodo COS e Densità via FFT

**Opzione:** Put europea

| Parametro | Valore |
|-----------|--------|
| $S_0$ | 100 |
| $K$ | 50, 100, 150 |
| $r$ | 10% |
| $q$ | 0 |
| $T$ | 0.5 anni |

**Modello CGMYB** (CGMY + moto Browniano):

| Parametro | Valore | Significato |
|-----------|--------|-------------|
| $\sigma$ | 0.2 | Volatilità Browniana |
| $C$ | 1 | Intensità dei salti |
| $G$ | 5 | Decadimento esponenziale salti negativi |
| $M$ | 5 | Decadimento esponenziale salti positivi |
| $Y$ | 0.5 | Indice di attività dei salti |

*Nota: $Y = 0.5$ come da Oosterlee & Grzelak (2019), pag. 248.*

## 1. Funzione caratteristica del modello CGMYB

Il log-rendimento $X_T = \ln(S_T/S_0)$ ha funzione caratteristica:

$$\varphi(u) = \exp\Big(iu(r - q + \omega)T - \tfrac{1}{2}\sigma^2 u^2 T + T\,\psi_{CGMY}(u)\Big)$$

dove l'esponente caratteristico CGMY è:

$$\psi_{CGMY}(u) = C\,\Gamma(-Y)\Big[(M - iu)^Y - M^Y + (G + iu)^Y - G^Y\Big]$$

e la correzione martingala: $\omega = -\psi_{CGMY}(-i) - \sigma^2/2$

In [ ]:
import numpy as np
from scipy.special import gamma as Gamma
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 12

In [ ]:
# Parametri di mercato
S0 = 100.0
r  = 0.1
q  = 0.0
T  = 0.5
strikes = [50.0, 100.0, 150.0]

# Parametri CGMYB
sigma_B = 0.2
C_par   = 1.0
G_par   = 5.0
M_par   = 5.0
Y_par   = 0.5

In [ ]:
def cgmy_exponent(u):
    """Esponente caratteristico del processo CGMY."""
    return C_par * Gamma(-Y_par) * (
        (M_par - 1j*u)**Y_par - M_par**Y_par +
        (G_par + 1j*u)**Y_par - G_par**Y_par
    )

# Correzione martingala
omega = -cgmy_exponent(-1j).real - 0.5 * sigma_B**2

def char_fn(u):
    """CF di X_T = ln(S_T/S_0) sotto misura risk-neutral."""
    return np.exp(
        1j * u * (r - q + omega) * T
        - 0.5 * sigma_B**2 * u**2 * T
        + T * cgmy_exponent(u)
    )

# Verifica: phi(0) deve valere 1
print(f"phi(0) = {char_fn(0.0):.6f} (deve essere 1.0)")
print(f"omega  = {omega:.6f}")

## 2. Intervallo di troncamento via cumulanti

Seguendo Fang & Oosterlee (2008):

$$[a,\, b] = \Big[c_1 - L\sqrt{c_2 + \sqrt{|c_4|}},\;\; c_1 + L\sqrt{c_2 + \sqrt{|c_4|}}\Big]$$

dove $c_n$ è il $n$-esimo cumulante di $X_T$.

In [ ]:
# Cumulanti CGMY
c1_cgmy = C_par * T * Gamma(1-Y_par) * (M_par**(Y_par-1) - G_par**(Y_par-1))
c2_cgmy = C_par * T * Gamma(2-Y_par) * (M_par**(Y_par-2) + G_par**(Y_par-2))
c4_cgmy = C_par * T * Gamma(4-Y_par) * (M_par**(Y_par-4) + G_par**(Y_par-4))

# Cumulanti totali (CGMY + Browniano + drift)
c1 = (r - q + omega)*T + c1_cgmy
c2 = sigma_B**2 * T + c2_cgmy
c4 = c4_cgmy

L_cos = 10
a_cos = c1 - L_cos * np.sqrt(c2 + np.sqrt(abs(c4)))
b_cos = c1 + L_cos * np.sqrt(c2 + np.sqrt(abs(c4)))

print(f"c1 = {c1:.6f}  (media)")
print(f"c2 = {c2:.6f}  (varianza)")
print(f"c4 = {c4:.6f}  (quarto cumulante)")
print(f"\nIntervallo di troncamento: [{a_cos:.4f}, {b_cos:.4f}]")

## 3. Metodo COS per il pricing della Put

Il prezzo della put con il metodo COS è:

$$P = K e^{-rT} \sum_{k=0}^{N-1}{}' \text{Re}\Big[\varphi\Big(\frac{k\pi}{b-a}\Big) e^{ik\pi\frac{x-a}{b-a}}\Big]\, H_k$$

dove $x = \ln(S_0/K)$, il $'$ indica che il primo termine è dimezzato, e i coefficienti della put sono:

$$H_k = \frac{2}{b-a}\Big[\psi_k(a, 0) - \chi_k(a, 0)\Big]$$

In [ ]:
def chi_k(c, d, k, a, b):
    """Integrale di e^y cos(k*pi*(y-a)/(b-a)) dy su [c,d]."""
    if k == 0:
        return np.exp(d) - np.exp(c)
    w = k * np.pi / (b - a)
    return (1.0/(1.0 + w**2)) * (
        np.cos(w*(d-a))*np.exp(d) - np.cos(w*(c-a))*np.exp(c)
        + w*(np.sin(w*(d-a))*np.exp(d) - np.sin(w*(c-a))*np.exp(c))
    )

def psi_k(c, d, k, a, b):
    """Integrale di cos(k*pi*(y-a)/(b-a)) dy su [c,d]."""
    if k == 0:
        return d - c
    return (b-a)/(k*np.pi) * (
        np.sin(k*np.pi*(d-a)/(b-a)) - np.sin(k*np.pi*(c-a)/(b-a))
    )

def cos_put_price(S0, K, T, r, N_cos, a, b):
    """Prezzo put europea con metodo COS."""
    x = np.log(S0 / K)
    k = np.arange(N_cos)
    u_k = k * np.pi / (b - a)

    # CF
    cf = char_fn(u_k)

    # Coefficienti payoff put
    Hk = np.array([
        (2.0/(b-a)) * (psi_k(a, 0.0, j, a, b) - chi_k(a, 0.0, j, a, b))
        for j in range(N_cos)
    ])

    # Somma COS (primo termine dimezzato)
    summand = np.real(cf * np.exp(1j * k * np.pi * (x-a)/(b-a))) * Hk
    summand[0] *= 0.5

    return K * np.exp(-r*T) * np.sum(summand)

## 4. Convergenza del metodo COS

In [ ]:
# Prezzo di riferimento con N molto grande
ref_prices = {K: cos_put_price(S0, K, T, r, 2**14, a_cos, b_cos) for K in strikes}
print("Prezzi di riferimento (N = 16384):")
for K, p in ref_prices.items():
    print(f"  K = {K:5.0f}: Put = {p:.8f}")

N_values = [4, 8, 16, 32, 64, 128, 256, 512]

results = []
for N in N_values:
    row = {'N': N}
    for K in strikes:
        p = cos_put_price(S0, K, T, r, N, a_cos, b_cos)
        row[f'Put K={int(K)}'] = p
        row[f'Err K={int(K)}'] = abs(p - ref_prices[K])
    results.append(row)

df = pd.DataFrame(results)
print("\nConvergenza COS:")
print(df.to_string(index=False, float_format=lambda x: f'{x:.2e}' if x < 0.1 else f'{x:.6f}'))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

for K in strikes:
    errs = [abs(cos_put_price(S0, K, T, r, N, a_cos, b_cos) - ref_prices[K]) for N in N_values]
    # Filtra errori zero per il log plot
    errs_plot = [max(e, 1e-16) for e in errs]
    axes[0].semilogy(N_values, errs_plot, 'o-', label=f'K={int(K)}')

axes[0].set_xlabel('N (termini espansione COS)')
axes[0].set_ylabel('|Errore|')
axes[0].set_title('Convergenza del metodo COS')
axes[0].legend(); axes[0].grid(True, which='both', alpha=0.3)

for K in strikes:
    prices_plot = [cos_put_price(S0, K, T, r, N, a_cos, b_cos) for N in N_values]
    axes[1].plot(N_values, prices_plot, 'o-', label=f'K={int(K)}')
    axes[1].axhline(ref_prices[K], ls='--', alpha=0.3)

axes[1].set_xlabel('N'); axes[1].set_ylabel('Prezzo Put')
axes[1].set_title('Convergenza del prezzo')
axes[1].legend(); axes[1].grid(alpha=0.3)

plt.tight_layout(); plt.show()

### Ruolo del parametro $L$ nel troncamento

Il parametro $L$ controlla l'ampiezza dell'intervallo $[a, b]$. Vediamo il suo effetto.

In [ ]:
print(f"{'L':>4} | {'[a, b]':>22} | {'Put K=100':>12} | {'Errore':>10}")
print('-'*60)

for L_test in [4, 6, 8, 10, 12, 15]:
    a_t = c1 - L_test*np.sqrt(c2 + np.sqrt(abs(c4)))
    b_t = c1 + L_test*np.sqrt(c2 + np.sqrt(abs(c4)))
    p = cos_put_price(S0, 100.0, T, r, 256, a_t, b_t)
    print(f"{L_test:4d} | [{a_t:+.4f}, {b_t:+.4f}] | {p:12.8f} | {abs(p-ref_prices[100]):.2e}")

## 5. Densità via inversione FFT

La densità di $X_T = \ln(S_T/S_0)$ si recupera dalla funzione caratteristica:

$$f(x) = \frac{1}{\pi}\,\text{Re}\int_0^{\infty} e^{-iux}\,\varphi(u)\,du$$

Approssimando con la regola dei trapezi e usando la FFT:

$$f(x_k) \approx \frac{\Delta u}{\pi}\,\text{Re}\Big[\text{FFT}\Big(\varphi(j\Delta u)\cdot e^{-ij\Delta u\, x_{min}}\Big)\Big]$$

con $\Delta x \cdot \Delta u = 2\pi / N_{FFT}$.

In [ ]:
def fft_density(char_fn, N_fft, du):
    """Recupera la densità via inversione FFT della CF."""
    u = np.arange(N_fft) * du
    x_min = -np.pi / du
    dx = 2*np.pi / (N_fft * du)
    x = x_min + np.arange(N_fft) * dx

    phi_vals = char_fn(u)
    phi_vals[0] *= 0.5  # peso trapezoidale al primo punto

    integrand = phi_vals * np.exp(-1j * u * x_min) * du
    density = np.real(np.fft.fft(integrand)) / np.pi

    return x, density

In [ ]:
N_fft = 2**14
du = 0.01

x_dens, f_cgmyb = fft_density(char_fn, N_fft, du)

# Densità normale con stessa media e varianza
f_normal = norm.pdf(x_dens, loc=c1, scale=np.sqrt(c2))

# Verifica integrazione
dx_grid = x_dens[1] - x_dens[0]
print(f"Integrale densità CGMYB: {np.sum(f_cgmyb)*dx_grid:.6f} (deve essere ~1)")
print(f"Media CGMYB:   {c1:.6f}")
print(f"Varianza CGMYB: {c2:.6f}")

## 6. Confronto densità CGMYB vs Normale

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

mask = (x_dens > -1.5) & (x_dens < 1.5)

axes[0].plot(x_dens[mask], f_cgmyb[mask], 'b-', lw=2, label='CGMYB (FFT)')
axes[0].plot(x_dens[mask], f_normal[mask], 'r--', lw=1.5, label='Normale')
axes[0].set_xlabel('$x = \ln(S_T/S_0)$')
axes[0].set_ylabel('Densità $f(x)$')
axes[0].set_title('Densità risk-neutral: CGMYB vs Normale')
axes[0].legend(); axes[0].grid(alpha=0.3)

# Log scale per evidenziare le code
mask2 = (x_dens > -3) & (x_dens < 3) & (f_cgmyb > 1e-8)
axes[1].semilogy(x_dens[mask2], f_cgmyb[mask2], 'b-', lw=2, label='CGMYB (FFT)')
axes[1].semilogy(x_dens[mask2], f_normal[mask2], 'r--', lw=1.5, label='Normale')
axes[1].set_xlabel('$x = \ln(S_T/S_0)$')
axes[1].set_ylabel('Densità (log scale)')
axes[1].set_title('Code della distribuzione')
axes[1].legend(); axes[1].grid(True, which='both', alpha=0.3)

plt.tight_layout(); plt.show()

### Ruolo dei parametri FFT: $N_{FFT}$ e $\Delta u$

In [ ]:
print("Effetto di du (N_fft = 2^14 fisso):")
print(f"{'du':>8} | {'dx':>8} | {'x_min':>8} | {'Integrale':>10}")
print('-'*45)
for du_test in [0.1, 0.05, 0.01, 0.005, 0.001]:
    x_t, f_t = fft_density(char_fn, 2**14, du_test)
    dx_t = x_t[1]-x_t[0]
    integ = np.sum(f_t)*dx_t
    print(f"{du_test:8.3f} | {dx_t:8.4f} | {x_t[0]:8.1f} | {integ:10.6f}")

print("\nEffetto di N_fft (du = 0.01 fisso):")
print(f"{'N_fft':>8} | {'dx':>8} | {'Integrale':>10}")
print('-'*35)
for n_exp in [10, 12, 14, 16]:
    N_t = 2**n_exp
    x_t, f_t = fft_density(char_fn, N_t, 0.01)
    dx_t = x_t[1]-x_t[0]
    integ = np.sum(f_t)*dx_t
    print(f"{N_t:8d} | {dx_t:8.4f} | {integ:10.6f}")

## 7. Tabella riassuntiva prezzi

In [ ]:
print("Prezzi Put europee — Modello CGMYB (N_COS = 256)")
print(f"S0={S0}, r={r}, q={q}, T={T}")
print(f"sigma={sigma_B}, C={C_par}, G={G_par}, M={M_par}, Y={Y_par}\n")

print(f"{'K':>6} | {'Put Price':>12} | {'Moneyness':>12}")
print('-'*38)
for K in strikes:
    p = cos_put_price(S0, K, T, r, 256, a_cos, b_cos)
    moneyness = 'Deep ITM' if K > S0*1.1 else ('ITM' if K > S0 else ('ATM' if K == S0 else ('OTM' if K > S0*0.7 else 'Deep OTM')))
    print(f"{K:6.0f} | {p:12.6f} | {moneyness:>12}")

## 8. Commenti

### Convergenza del metodo COS

- Il metodo COS converge **molto rapidamente** (esponenzialmente): già con $N \approx 64$-128 termini l'errore è trascurabile.
- La convergenza è più lenta per strike estremi (K=50 deep OTM, K=150 deep ITM) perché il payoff è concentrato nelle code.
- Il parametro critico è **$L$** (ampiezza dell'intervallo $[a,b]$): se $L$ è troppo piccolo, si tronca la densità e il prezzo è impreciso; se troppo grande, servono più termini $N$ per la stessa accuratezza.
- La scelta $L = 10$ con la formula dei cumulanti garantisce ottima precisione.

### Densità CGMYB vs Normale

- La densità CGMYB ha un **picco più alto e stretto** della normale (leptocurtica).
- Le **code sono più pesanti**: nel grafico in scala logaritmica si vede che la densità CGMYB rimane sopra la normale per $|x|$ grande. Questo è dovuto ai salti del processo CGMY.
- Poiché $G = M = 5$ (simmetria), la distribuzione è circa simmetrica, come la normale.
- Le code pesanti spiegano perché i prezzi delle opzioni deep OTM/ITM sono più alti rispetto a un modello puramente gaussiano.

### Parametri critici FFT

- **$\Delta u$ (passo nella frequenza)**: controlla l'ampiezza del dominio spaziale ($x_{min} = -\pi/\Delta u$). Se $\Delta u$ è troppo grande, il dominio è troppo piccolo e la densità viene "tagliata".
- **$N_{FFT}$**: controlla la risoluzione $\Delta x = 2\pi/(N_{FFT} \cdot \Delta u)$. Un $N$ più grande dà una griglia più fine ma è più costoso.
- I valori $\Delta u = 0.01$ e $N = 2^{14}$ danno un buon compromesso.